# Running BOA Optimization Directly in Python

This notebook demonstrates how to:

Write a basic Wrapper in Python and launch a optimization from Python.
If you wanted to launch it from command line, you would do a similar thing of defining the Wrapper, and then put in your configuration file the information about where the wrapper is, and use [BOA's](../index.rst) CLI tools. See [Running an Experiment from Command Line (Python Wrapper)](example_py_run.rst) for more information.

In [1]:
import pathlib

import yaml
import shutil

import boa
from wrapper import Wrapper

In [2]:
# Remove old runs to have a clean slate for this example
old_runs = pathlib.Path().resolve().glob("boa_runs*")
for path in old_runs:
    shutil.rmtree(path)

## Loading the Config File

In [3]:
config_path = pathlib.Path().resolve() / "single_config.yaml"

Here we can see what the configuration file looks like

In [4]:
with open(config_path, 'r') as f:
    file_contents = f.read()
    print (file_contents)

# Single objective optimization config
optimization_options:
    objective_options:
        objectives:
            - name: Cosine8
    trials: 50
    append_timestamp: False
parameters:
    x0:
        type: range
        bounds: [0.0, 1.0]
    x1:
        type: range
        bounds: [0.0, 1.0]
    x2:
        type: range
        bounds: [0.0, 1.0]
    x3:
        type: range
        bounds: [0.0, 1.0]
    x4:
        type: range
        bounds: [0.0, 1.0]
    x5:
        type: range
        bounds: [0.0, 1.0]
    x6:
        type: range
        bounds: [0.0, 1.0]
    x7:
        type: range
        bounds: [0.0, 1.0]
# These are all defaults, so we don't need to specify them in this case
#script_options:
#    wrapper_path: ./wrapper.py
#    wrapper_name: Wrapper
#    working_dir: .
#    experiment_dir: ... # this is where boa will write logs to by default
                         # if not specified it will be working_dir/experiment_name
#    append_timestamp: True
# This last option 

we need the config normalized, which modifies the parameter section
into a less user friendly form, but what the downstream libraries need

In [5]:
config = boa.load_jsonlike(config_path)

## Define Our Wrapper

We define our wrapper in wrapper.py and use a synthetic function that stands in for any black box model call

In [6]:
Wrapper.path()

PosixPath('/Users/madelinescyphers/Documents/projs_.nosync/boa/docs/examples/wrapper.py')

In [7]:
with open(Wrapper.path(), 'r') as f:
    file_contents = f.read()
    print (file_contents)

import numpy as np
from ax.utils.measurement.synthetic_functions import from_botorch
from botorch.test_functions.synthetic import Cosine8

import boa

cosine8 = from_botorch(Cosine8())


def black_box_model(X) -> float:
    result = -cosine8(X)
    return result


class Wrapper(boa.BaseWrapper):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.data = {}

    def run_model(self, trial) -> None:
        X = np.array([parameter for parameter in trial.arm.parameters.values()])
        # This is a silly toy function, in reality,
        # you could instead import your model main() function and use that, and then collect the results.
        # You could also call an external script to start a model run from Bash or elsewhere.
        self.data[trial.index] = black_box_model(X)

    def set_trial_status(self, trial) -> None:
        data_exists = self.data.get(trial.index)
        if data_exists:
            trial.mark_completed()

    def fetch_

## Initialize our Setup

In [8]:
controller = boa.Controller(config_path=config_path, wrapper=Wrapper)

controller.initialize_scheduler()

[INFO 07-09 15:33:04] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x0. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-09 15:33:04] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-09 15:33:04] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-09 15:33:04] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x3. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-09 

(Scheduler(experiment=Experiment(boa_runs), generation_strategy=GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 16 trials, GPEI for subsequent trials]), options=SchedulerOptions(max_pending_trials=10, trial_type=<TrialType.TRIAL: 0>, batch_size=None, total_trials=None, tolerated_trial_failure_rate=0.5, min_failed_trials_for_failure_rate_check=5, log_filepath=None, logging_level=20, ttl_seconds_for_trials=None, init_seconds_between_polls=1, min_seconds_before_poll=1.0, seconds_between_polls_backoff_factor=1.5, timeout_hours=None, run_trials_in_batches=False, debug_log_run_metadata=False, early_stopping_strategy=None, global_stopping_strategy=None, suppress_storage_errors_after_retries=False)),
 <wrapper.Wrapper at 0x156b75bd0>)

## Run our Experiment

The Controller will save our scheduler to JSON after it completes the run so we can reload it at a later time for analysis or to resume our experiment

In [9]:
scheduler = controller.run()

[INFO 2023-07-09 15:33:04,686 MainProcess] boa: 

##############################################


BOA Experiment Run
Output Experiment Dir: /Users/madelinescyphers/Documents/projs_.nosync/boa/docs/examples/boa_runs_20230709T153304
Start Time: 20230709T153304
Version: 0.8.3.dev1+g0636f51.d20230411

##############################################

[INFO 07-09 15:33:04] Scheduler: Running trials [0]...
[INFO 07-09 15:33:05] Scheduler: Running trials [1]...
[INFO 07-09 15:33:06] Scheduler: Running trials [2]...
[INFO 07-09 15:33:07] Scheduler: Running trials [3]...
[INFO 07-09 15:33:08] Scheduler: Running trials [4]...
[INFO 07-09 15:33:09] Scheduler: Running trials [5]...
[INFO 07-09 15:33:10] Scheduler: Running trials [6]...
[INFO 07-09 15:33:11] Scheduler: Running trials [7]...
[INFO 07-09 15:33:12] Scheduler: Running trials [8]...
[INFO 07-09 15:33:13] Scheduler: Running trials [9]...
[INFO 07-09 15:33:15] Scheduler: Retrieved COMPLETED trials: 0 - 9.
[INFO 07-09 15:33:15] Scheduler: F

## Get the Best Trial and Output All Trials

In [11]:
trial = scheduler.best_raw_trials()
trial

{49: {'params': {'x0': 0.38758168014208244,
   'x1': 0.13726610247163673,
   'x2': 0.41894905055994475,
   'x3': 0.0,
   'x4': 0.36887117986295076,
   'x5': 0.0,
   'x6': 0.0,
   'x7': 0.0},
  'means': {'Cosine8': -0.1460910190948791},
  'cov_matrix': {'Cosine8': {'Cosine8': 0.0}}}}

In [12]:
boa.scheduler_to_df(scheduler)

,trial_index,arm_name,trial_status,generation_method,Cosine8,x0,x1,x2,x3,x4,x5,x6,x7
0,0,0_0,COMPLETED,Sobol,1.524703,0.186015,0.272769,0.016171,0.266545,0.560669,0.513949,0.416432,0.651428
1,1,1_0,COMPLETED,Sobol,3.039584,0.340813,0.315056,0.438800,0.113317,0.968783,0.986203,0.829010,0.133948
2,2,2_0,COMPLETED,Sobol,2.263460,0.393835,0.234749,0.470373,0.902659,0.719156,0.544104,0.034133,0.523170
3,3,3_0,COMPLETED,Sobol,4.074467,0.056779,0.708886,0.955217,0.504353,0.904291,0.525549,0.613254,0.891620
4,4,4_0,COMPLETED,Sobol,2.323068,0.419098,0.406458,0.389148,0.108567,0.640533,0.358138,0.985209,0.692024
5,5,5_0,COMPLETED,Sobol,3.154856,0.529539,0.590154,0.865609,0.360900,0.882468,0.645021,0.002805,0.685443
6,6,6_0,COMPLETED,Sobol,1.538552,0.931053,0.091655,0.465270,0.200827,0.556495,0.040644,0.018876,0.277865
7,7,7_0,COMPLETED,Sobol,1.186934,0.035618,0.649624,0.423637,0.654493,0.378146,0.106575,0.404570,0.403155
8,8,8_0,COMPLETED,Sobol,2.754577,0.582191,0.939444,0.796483,0.042948,0.504899,0.812377,0.141764,0.102854
9,9,9_0,COMPLETED,Sobol,2.583942,0.095407,0.215372,0.691654,0.320616,0.992338,0.242350,0.482226,0.592037
